In [4]:
import org.apache.spark.graphx.{Graph, GraphLoader, VertexId, VertexRDD}
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}

import org.apache.spark.graphx.{Graph, GraphLoader, VertexId, VertexRDD}
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}


In [5]:
class Graph[VD, ED] {
  val vertices: VertexRDD[VD]
  val edges: EdgeRDD[ED]
}

<console>: 13: error: not found: type VertexRDD

In [2]:
/**
   * An RDD containing the vertices and their associated attributes.
   *
   * @note vertex ids are unique.
   * @return an RDD containing the vertices in this graph
   */
  val vertices: VertexRDD[VD]

  /**
   * An RDD containing the edges and their associated attributes.  The entries in the RDD contain
   * just the source id and target id along with the edge data.
   *
   * @return an RDD containing the edges in this graph
   *
   * @see `Edge` for the edge type.
   * @see `Graph#triplets` to get an RDD which contains all the edges
   * along with their vertex data.
   *
   */
  val edges: EdgeRDD[ED]

  /**
   * An RDD containing the edge triplets, which are edges along with the vertex data associated with
   * the adjacent vertices. The caller should use [[edges]] if the vertex data are not needed, i.e.
   * if only the edge data and adjacent vertex ids are needed.
   *
   * @return an RDD containing edge triplets
   *
   * @example This operation might be used to evaluate a graph
   * coloring where we would like to check that both vertices are a
   * different color.
   * {{{
   * type Color = Int
   * val graph: Graph[Color, Int] = GraphLoader.edgeListFile("hdfs://file.tsv")
   * val numInvalid = graph.triplets.map(e => if (e.src.data == e.dst.data) 1 else 0).sum
   * }}}
   */
  val triplets: RDD[EdgeTriplet[VD, ED]]

  /**
   * Caches the vertices and edges associated with this graph at the specified storage level,
   * ignoring any target storage levels previously set.
   *
   * @param newLevel the level at which to cache the graph.
   *
   * @return A reference to this graph for convenience.
   */


<console>: 32: error: not found: type VertexRDD

In [ ]:
val sc: SparkContext
val users: RDD[(VertexId, (String, String))] =
sc.parallelize(Array(
                        (3L, ("rxin", "student")), 
                        (7L, ("jgonzal", "postdoc")),
                        (5L, ("franklin", "prof")), 
                        (2L, ("istoica", "prof"))
            ))
val relationships: RDD[Edge[String]] = sc.parallelize(Array(
                    Edge(3L, 7L, "collab"),    
                    Edge(5L, 3L, "advisor"),
                    Edge(2L, 5L, "colleague"), 
                    Edge(5L, 7L, "pi")
                ))
val defaultUser = ("John Doe", "Missing")

val graph = Graph(users, relationships, defaultUser)




In [ ]:
object GraphTest {
  def main(args: Array[String]): Unit = {
    val conf: SparkConf = new SparkConf().setAppName(this.getClass.getSimpleName).setMaster("local")
    val sc: SparkContext = new SparkContext(conf)
    //读取followers.txt文件创建图
    val graph: Graph[Int, Int] = GraphLoader.edgeListFile(sc,"F:\\followers.txt")
    //计算连通体
    val components: Graph[VertexId, Int] = graph.connectedComponents()
    val vertices: VertexRDD[VertexId] = components.vertices
    /**
      * vertices：
      * (4,1)
      * (1,1)
      * (6,3)
      * (3,3)
      * (7,3)
      * (2,1)
      * 是一个tuple类型，key分别为所有的顶点id，value为key所在的连通体id(连通体中顶点id最小值)
      */
    //读取users.txt文件转化为(key,value)形式
    val users: RDD[(VertexId, String)] = sc.textFile("F:\\users.txt").map(line => {
      val fields: Array[String] = line.split(",")
      (fields(0).toLong, fields(1))
    })
    /**
      * users:
      * (1,BarackObama)
      * (2,ladygaga)
      * (3,jeresig)
      * (4,justinbieber)
      * (6,matei_zaharia)
      * (7,odersky)
      * (8,anonsys)
      */
    users.join(vertices).map{
      case(id,(username,vertices))=>(vertices,username)
    }.groupByKey().map(t=>{
      t._1+"->"+t._2.mkString(",")
    }).foreach(println(_))
    /**
      * 得到结果为：
      * 1->justinbieber,BarackObama,ladygaga
      * 3->matei_zaharia,jeresig,odersky
      */
  }
 
}